# Генерация координат квартир

## Импорт библиотек

In [60]:
import requests

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import re

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

## Открытие данных

In [61]:
df = pd.read_csv('../clean_categorizated_data_without_outliers.csv', sep=';')

In [62]:
df.sample(3)

,number of rooms,area of apartment,number of floors,apartment floor,price,year of construction,elevator,full address,concierge,garbage chute,...,district_Индустриальный,district_Кировский,district_Ленинский,district_Мотовилихинский,district_Орджоникидзевский,district_Свердловский,parking_за шлагбаумом во дворе,parking_открытая во дворе,parking_наземная многоуровневая,parking_подземная
1663,1,40.3,16,10,3800000.0,2001,1.0,"Пермский край, Пермь, ул. Толмачёва, 17",1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
530,2,45.0,17,7,4080000.0,2021,1.0,"Пермский край, Пермь, ул. Овчинникова, 15А",1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,3,76.8,10,5,3550000.0,2016,2.0,"Пермский край, Пермь, Молдавская ул., 10А",0,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   number of rooms                  1894 non-null   int64  
 1   area of apartment                1894 non-null   float64
 2   number of floors                 1894 non-null   int64  
 3   apartment floor                  1894 non-null   int64  
 4   price                            1894 non-null   float64
 5   year of construction             1894 non-null   int64  
 6   elevator                         1894 non-null   float64
 7   full address                     1894 non-null   object 
 8   concierge                        1894 non-null   int64  
 9   garbage chute                    1894 non-null   int64  
 10  repair_дизайнерский              1894 non-null   float64
 11  repair_евро                      1894 non-null   float64
 12  repair_косметический

## Получение координат дома квартиры

In [72]:
geo_coder_url = "https://geotree.ru/geocoder"

In [102]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\Daniil\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [103]:
coords = np.zeros(shape=(len(df), 2))
no_coords_dict = dict()

In [104]:
for idx, address in enumerate(df['full address']):
    if idx%100 == 0:
        print(f"{idx}/{len(df)}")
    try:
        # Обращаемся к сайту-геокедру
        driver.get(geo_coder_url)
        # Находим элемент для вставки адреса и помещаем туда адрес
        input_address = driver.find_element_by_id("input_address")
        input_address.clear()
        input_address.send_keys(address)
        time.sleep(1)
        # Извлекаем результат и преобразуем в словарь
        result = driver.find_element_by_id("div_result_json").find_element_by_tag_name('pre')
        text = result.text.replace('\n', '')
        info = json.loads(text)
        # Достаем из словаря координаты объекта
        lat = info[0].get('geo_center').get('lat')
        lon = info[0].get('geo_center').get('lon')
        # Сохраняем координаты
        coords[idx][0] = lat
        coords[idx][1] = lon
    except Exception as ex:
        print("Обнаружена ошибка с адресом номер " + str(idx) + " " + address)
        print(ex)
        if address in no_coords_dict:
            no_coords_dict[address].append(idx)
        else:
            no_coords_dict[address] = [idx]
        print(no_coords_dict[address])
        print("--------------------")
        continue

0/1894
Обнаружена ошибка с адресом номер 3 Пермский край, Пермь, Красногвардейская ул., 2
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[3]
--------------------
Обнаружена ошибка с адресом номер 12 Пермский край, Пермь, Корсуньская ул., 29
list index out of range
[12]
--------------------
Обнаружена ошибка с адресом номер 19 Пермский край, Пермь, Красногвардейская ул., 2
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[3, 19]
--------------------
Обнаружена ошибка с адресом номер 23 Пермский край, Пермь, Корсуньская ул., 29
list index out of range
[12, 23]
--------------------
Обнаружена ошибка с адресом номер 36 Пермский край, Пермь, ул. Фридриха Энгельса, 27
list index out of range
[36]
--------------------
Обнаружена ошибка с адресом номер 38 Пермский край, Пермь, Красногвардейская ул., 2
Message: no such elemen

400/1894
Обнаружена ошибка с адресом номер 406 Пермский край, Пермь, ул. Веры Фигнер, 2
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[406]
--------------------
Обнаружена ошибка с адресом номер 409 Пермский край, Пермь, Чистопольская ул., 15
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[409]
--------------------
Обнаружена ошибка с адресом номер 410 Пермский край, Пермь, ул. Старцева, 5
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[410]
--------------------
Обнаружена ошибка с адресом номер 412 Пермский край, Пермь, Можайская ул., 20
list index out of range
[412]
--------------------
Обнаружена ошибка с адресом номер 419 Пермский край, Пермь, Молдавская ул., 10А
list index out of range
[126, 293, 419]
--------------------
О

Обнаружена ошибка с адресом номер 872 Пермский край, Пермь, ул. Голева, 3
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[542, 672, 872]
--------------------
Обнаружена ошибка с адресом номер 883 Пермский край, Пермь, ул. Овчинникова, 15А
list index out of range
[351, 530, 883]
--------------------
Обнаружена ошибка с адресом номер 893 Пермский край, Пермь, Теплогорская ул., 24к3
list index out of range
[893]
--------------------
900/1894
Обнаружена ошибка с адресом номер 905 Пермский край, Пермь, Серпуховская ул., 8
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[905]
--------------------
Обнаружена ошибка с адресом номер 908 Пермский край, Пермь, Бершетский пер., 6
list index out of range
[908]
--------------------
Обнаружена ошибка с адресом номер 909 Пермский край, Пермь, Серединная ул., 2
Message: no such elem

Обнаружена ошибка с адресом номер 1161 Пермский край, Пермь, ул. Овчинникова, 15А
list index out of range
[351, 530, 883, 1161]
--------------------
Обнаружена ошибка с адресом номер 1171 Пермский край, Пермь, ул. Моторостроителей, 3
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[1171]
--------------------
Обнаружена ошибка с адресом номер 1181 Пермский край, Пермь, ул. Желябова, 3
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[1049, 1181]
--------------------
Обнаружена ошибка с адресом номер 1189 Пермский край, Пермь, ул. Подводников, 4
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[1189]
--------------------
Обнаружена ошибка с адресом номер 1193 Пермский край, Пермь, Малая ул., 1
Message: no such element: Unable to locate 

Обнаружена ошибка с адресом номер 1447 Пермский край, Пермь, Кустовая ул., 1
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[1447]
--------------------
Обнаружена ошибка с адресом номер 1448 Пермский край, Пермь, ул. Космонавта Беляева, 8
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[1448]
--------------------
Обнаружена ошибка с адресом номер 1449 Пермский край, Пермь, ул. Оборонщиков, 3
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[1449]
--------------------
Обнаружена ошибка с адресом номер 1465 Пермский край, Пермь, Красногвардейская ул., 2
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[3, 19, 38, 48, 54, 96, 157, 366, 675, 961, 1465]


Обнаружена ошибка с адресом номер 1751 Пермский край, Пермь, Весенняя ул., 3
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[1751]
--------------------
Обнаружена ошибка с адресом номер 1772 Пермский край, Пермь, ул. Евгения Пермяка, 1
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[1252, 1591, 1772]
--------------------
Обнаружена ошибка с адресом номер 1773 Пермский край, Пермь, 1-й Еловский пер., 22
list index out of range
[1773]
--------------------
Обнаружена ошибка с адресом номер 1774 Пермский край, Пермь, Делегатская ул., 39Б
list index out of range
[273, 1774]
--------------------
Обнаружена ошибка с адресом номер 1775 Пермский край, Пермь, Подлесная ул., 2
Message: no such element: Unable to locate element: {"method":"css selector","selector":"pre"}
  (Session info: chrome=96.0.4664.45)

[768, 1544, 1775]

In [105]:
driver.close()

## Вручную дополняем самые распространенные адреса

In [118]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\Daniil\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [126]:
for address in no_coords_dict:
    if not no_coords_dict[address]:
        continue
    elif len(no_coords_dict[address]) >= 2:
        try:
            # Обращаемся к сайту-геокедру
            driver.get(geo_coder_url)
            # Находим элемент для вставки адреса и помещаем туда адрес
            input_address = driver.find_element_by_id("input_address")
            input_address.clear()
            input_address.send_keys(address)
            time.sleep(2)
            # Извлекаем результат и преобразуем в словарь
            result = driver.find_element_by_id("div_result_json").find_element_by_tag_name('pre')
            text = result.text.replace('\n', '')
            info = json.loads(text)
            # Достаем из словаря координаты объекта
            lat = info[0].get('geo_center').get('lat')
            lon = info[0].get('geo_center').get('lon')
            # Сохраняем координаты
            for idx in no_coords_dict[address]:
                coords[idx][0] = lat
                coords[idx][1] = lon
            no_coords_dict[address] = None
        except Exception as ex:
            print("Обнаружена ошибка с адресом номер " + address)
            print(ex)
            string = input().split(', ')
            lat = float(string[0].strip())
            lon = float(string[1].strip())
            if lon == 0 and lat == 0:
                break
            for idx in no_coords_dict[address]:
                coords[idx][0] = lat
                coords[idx][1] = lon
            no_coords_dict[address] = None
            print("--------------------")
            continue

Обнаружена ошибка с адресом номер Пермский край, Пермь, Янаульская ул., 18
list index out of range
58.101668, 56.305989
--------------------
Обнаружена ошибка с адресом номер Пермский край, Пермь, Делегатская ул., 39Б
list index out of range
58.135157, 56.399450
--------------------
Обнаружена ошибка с адресом номер Пермский край, Пермь, Петропавловская ул., 13А
list index out of range
58.017582, 56.251892
--------------------
Обнаружена ошибка с адресом номер Пермский край, Пермь, 1-й Еловский пер., 24
list index out of range
58.079204, 56.362268
--------------------
Обнаружена ошибка с адресом номер Пермский край, Пермь, ул. Лодыгина, 46к2
list index out of range
57.965808, 56.217074
--------------------
Обнаружена ошибка с адресом номер Пермский край, Пермь, Ласьвинская ул., 56А
list index out of range
58.007742, 55.934032
--------------------
Обнаружена ошибка с адресом номер Пермский край, Пермь, Теплогорская ул., 24к3
list index out of range
57.973541, 56.188265
-----------------

In [127]:
driver.close()

## Сохраняем координаты в таблицу

In [138]:
lat = coords[:, 0].copy()
lon = coords[:, 1].copy()

In [141]:
lat.shape, lon.shape

((1894,), (1894,))

In [143]:
df['lat'] = lat
df['lon'] = lon

In [147]:
df = df.loc[df.lat != 0]

## Итоги генерации координат

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1767 entries, 0 to 1893
Data columns (total 38 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   number of rooms                  1767 non-null   int64  
 1   area of apartment                1767 non-null   float64
 2   number of floors                 1767 non-null   int64  
 3   apartment floor                  1767 non-null   int64  
 4   price                            1767 non-null   float64
 5   year of construction             1767 non-null   int64  
 6   elevator                         1767 non-null   float64
 7   full address                     1767 non-null   object 
 8   concierge                        1767 non-null   int64  
 9   garbage chute                    1767 non-null   int64  
 10  repair_дизайнерский              1767 non-null   float64
 11  repair_евро                      1767 non-null   float64
 12  repair_косметический

In [148]:
df.to_csv('../clean_categorizated_data_without_outliers_with_coordinates.csv', sep=';', index=False)

__В ходе четвертой обработки были:__
1. получены значения координат квартир